In [1]:
import os
import torch
import torch.nn as nn
from torch import optim

from cnn_framework.dummy_regression_cnn.data_set import DummyRegressionCnnDataSet
from cnn_framework.dummy_regression_cnn.model_params import DummyModelParams
from cnn_framework.dummy_regression_cnn.model import DummyCnn

from cnn_framework.utils.data_loader_generators.DataLoaderGenerator import DataLoaderGenerator
from cnn_framework.utils.model_managers.RegressionModelManager import RegressionModelManager
from cnn_framework.utils.DataManagers import DefaultDataManager
from cnn_framework.utils.metrics import MeanSquaredErrorMetric
from cnn_framework.utils.create_dummy_data_set import generate_data_set

In [2]:
params = DummyModelParams()
params.update()

# Create data set if needed
if not os.path.exists(params.data_dir):
    generate_data_set(params.data_dir)
    print(f"\nData set created in {params.data_dir}")

print(f"Model will be saved in {params.models_folder}")
print(f"Predictions will be saved in {params.output_dir}")
print(f"Tensorboard logs will be saved in {params.tensorboard_folder_path}")

Model time id: 20230901-115658-local
epochs 10 | batch 2 | lr 0.01 | weight decay 0.0 | dropout 0.0


In [3]:
loader_generator = DataLoaderGenerator(params, DummyRegressionCnnDataSet, DefaultDataManager)
train_dl, val_dl, test_dl = loader_generator.generate_data_loader()

### Data source ###
train data is loaded from C:\Users\thoma\data\Other\square_circle_uniform - 80% elements
val data is loaded from C:\Users\thoma\data\Other\square_circle_uniform - 10% elements
test data is loaded from C:\Users\thoma\data\Other\square_circle_uniform - 10% elements
###################
train has 160 images.
val has 20 images.
test has 20 images.
###################


In [4]:
model = DummyCnn(
    nb_classes=params.nb_classes,
    nb_input_channels=params.nb_modalities * params.nb_stacks_per_modality,
)

manager = RegressionModelManager(model, params, MeanSquaredErrorMetric)

optimizer = optim.Adam(
    model.parameters(), lr=float(params.learning_rate), betas=(params.beta1, params.beta2),
)  # define the optimization

loss_function = nn.L1Loss()  # define the loss function

Current commit hash: 0bd76adb62d6ab724499f3f60873108d76b52a28


In [5]:
manager.fit(train_dl, val_dl, optimizer, loss_function)

Training in progress: 100.0% | Local step 79 | Epoch 9
Best model saved at epoch 8.

Training successfully finished in 0:00:29.137767.


In [6]:
for model_path, name in zip(
    [manager.model_save_path_early_stopping, manager.model_save_path],
    ["early stopping", "final"],
):
    print(f"\nPredicting with {name} model.")
    # Update model with saved one
    manager.model.load_state_dict(torch.load(model_path))
    manager.predict(test_dl)


Predicting with early stopping model.
Model evaluation in progress: 100.0% | Batch #9
Average MeanSquaredError: -799.68

Predicting with final model.
Model evaluation in progress: 100.0% | Batch #9
Average MeanSquaredError: -1886.49
